# H₂ Molecule Homework Assignment
### Quantum Software Development Journey: From Theory to Application with Classiq - Part 3

- Similarly to what we have done in class, in this exercise we will implement the VQE on H2 molecule.
- This time instead of using the built-in methods and functions (such as `Molecule` and `MoleculeProblem`) to difne and solve the problem, you will be provided with a two qubits Hamiltonian.

## Submission
- Submit the completed Jupyter notebook and report via GitHub. Ensure all files are correctly named and organized.
- Use the Typeform link provided in the submission folder to confirm your submission.

## Additional Resources
- [Classiq Documentation](https://docs.classiq.io/latest/)
- The notebook from live session #3

## Important Dates
- **Assignment Release:** 22.5.2024
- **Submission Deadline:** 3.6.2024 (7 A.M GMT+3)

---

Happy coding and good luck!

### Part 1

Given the following Hamiltonian:

$$
\hat{H} = -1.0523 \cdot (I \otimes I) + 0.3979 \cdot (I \otimes Z) - 0.3979 \cdot (Z \otimes I) - 0.0112 \cdot (Z \otimes Z) + 0.1809 \cdot (X \otimes X)
$$

Complete the following code

In [2]:
from typing import List

from classiq import *

HAMILTONIAN = QConstant(
    "HAMILTONIAN",
    List[PauliTerm],
    [
        PauliTerm([Pauli.I, Pauli.I], -1.0523),
        PauliTerm([Pauli.I, Pauli.Z], 0.3979),
        PauliTerm([Pauli.Z, Pauli.I], -0.3979),
        PauliTerm([Pauli.Z, Pauli.Z], -0.0112),
        PauliTerm([Pauli.X, Pauli.X], 0.1809),
    ],
)

In [90]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,  # Constrained Optimization by Linear Approximation
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})


qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/4ab447f2-6a0b-4566-a399-e4f60086b54a?version=0.42.0


In [73]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value

In [74]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.0677845703125
Optimal parameters: {'angles_0': 6.082307593317608, 'angles_1': -3.7953740276822554, 'angles_2': -5.117916314867475}
Eigenstate: {'01': (0.09375+0j), '10': (0.09631896879639025+0j), '00': (0.9909256247317454+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class?

### Part 2

**Now, we want to have a more interesting ansatz in our `main`.**  
Add **one** line of code to the `main` function you created in Part 1 that creates **entanglement** between the two qubits.  
Which gate should you use?

In [3]:
from typing import List

from classiq import *


@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])  # Apply U gate to the first qubit
    U(angles[0], angles[1], angles[2], 0, q[1])  # Apply U gate to the second qubit
    # Entangling after the two U gates
    CX(q[0], q[1])  # Apply CNOT gate to create entanglement


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,  # Constrained Optimization by Linear Approximation
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})


qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/407f1b67-a0f0-4f58-b2c2-3e627a93e623?version=0.42.1


In [6]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value

In [7]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.862721484375
Optimal parameters: {'angles_0': 3.288397798297685, 'angles_1': 6.384016144975798, 'angles_2': 4.551714189035623}
Eigenstate: {'10': (0.06629126073623882+0j), '11': (0.08267972847076846+0j), '01': (0.9943689110435825+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class? \
What can we learn about the provided  form this result Hamiltonian?